In [1]:
# webbrowser : 브라우저로 특정 페이지를 여는 모듈
import webbrowser
# requests : 인터넷에서 파일과 웹 페이지를 다운로드 하는 모듈
import requests
# bs4 : HTML을 구문분석(parsing)하는 라이브러리
from bs4 import BeautifulSoup as BS
# selenium.webdriver : 웹 브라우저를 띄우고 제어하는 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
# urlopen으로 직접 열 수도 있는데.. requests로 다운받아놓는게 더 안전
from urllib.request import urlopen
import pyinputplus as pyip
from pathlib import Path

In [5]:
# webbroswer.open()함수 활용
# 주소의 법칙을 알면 pyinputplus 및 git bash(sys.argv[])와 연계 가능
findAddress = pyip.inputStr(prompt='찾는 주소를 입력해주세요.')
webbrowser.open(f'https://map.naver.com/p/search/{findAddress}?c=13.00,0,0,0,dh')

찾는 주소를 입력해주세요.

True